<a href="https://colab.research.google.com/github/barzansaeedpour/advanced-python-notebooks/blob/main/SOLID_Principles_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SOLID Principles with Python

## Resources

- https://medium.com/@m.nusret.ozates/solid-principles-with-python-245e45f9b1f8